In [82]:
from __future__ import print_function, division
import pprint
import os 
import fnmatch
#import errno
import os.path as osp
import json
#import re
import glob as gb
from six import string_types
from nipype.interfaces.base import Bunch

import utils
reload(utils)

<module 'utils' from 'utils.py'>

In [83]:
pp = pprint.PrettyPrinter(indent=2)

In [84]:
#- testing

test_case_105a = {
    'base_dir': '/home/jb/data/bids/ds105',
    'model_pattern': "ds-105_level-run_sub*_model.json"
}
test_case_105b = {
    'base_dir': '/home/jb/data/bids/ds105',
    'model_pattern': "ds-105_level-all_model.json"
}
test_case_005 = {
    'base_dir': '/home/jb/data/bids/ds005',
    'model_pattern': "*_model.json"
}
test_case_005r = {
    'base_dir': '/home/jb/data/bids/ds005',
    'model_pattern': "ds-005_type-russ*_model.json"
}

#test_case = test_case_005
test_case = test_case_005r
#test_case = test_case_105a
#test_case = test_case_105b

base_dir, model_pattern = test_case['base_dir'], test_case['model_pattern'] 

all_jsons = utils.glob_recursive(base_dir, model_pattern)
print(all_jsons)
json_model=all_jsons[0]

# data = utils.get_json_model_Ydata(all_jsons[0], level='Run', verbose=utils.VERB['info'])
model_data = utils.associate_model_data(base_dir, model_pattern, verbose=utils.VERB['none'])
model_dict = model_data.values()[0]
#print(model_dict)

['/home/jb/data/bids/ds005/models/ds-005_type-russ_sub-all_model.json']


In [85]:
model_pattern

'ds-005_type-russ*_model.json'

In [87]:
a, b, c = utils._get_nipype_specify_model_inputs(base_dir, model_pattern, bunch_type='fsl', 
                                                 verbose=utils.VERB['none'])
b[1]

Bunch(amplitudes=[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [15, 12, 15, 16, 5, 13, 17, 5, 7, 8, 10, 13, 17, 19, 20, 16, 5, 12, 16, 5, 20, 17, 6, 18, 7, 6, 20, 15, 6, 15, 17, 14, 13, 19, 12, 10, 17, 13, 13, 14, 11, 9, 16, 10, 8, 20, 16, 12, 12, 10, 7, 13, 19, 5, 9, 7, 13, 5, 9, 11, 14, 19, 7, 6, 18, 6, 12, 20, 17, 7, 20, 6, 6, 7, 10, 14, 19, 18, 12, 19, 9, 18, 12, 9, 12, 14], [2.419, 1.3580000000000001, 1.462, 2.1890000000000001, 2.5819999999999999, 2.823, 1.3660000000000001, 1.5519999999999998, 1.6120000000000001, 1.589, 1.885, 2.8849999999999998, 1.2629999999999999, 2.7010000000000001, 1.327, 1.448000

In [88]:
a, b, c = utils._get_nipype_specify_model_inputs(base_dir, model_pattern, bunch_type='fsl')
b[1]

Bunch(amplitudes=[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [15, 12, 15, 16, 5, 13, 17, 5, 7, 8, 10, 13, 17, 19, 20, 16, 5, 12, 16, 5, 20, 17, 6, 18, 7, 6, 20, 15, 6, 15, 17, 14, 13, 19, 12, 10, 17, 13, 13, 14, 11, 9, 16, 10, 8, 20, 16, 12, 12, 10, 7, 13, 19, 5, 9, 7, 13, 5, 9, 11, 14, 19, 7, 6, 18, 6, 12, 20, 17, 7, 20, 6, 6, 7, 10, 14, 19, 18, 12, 19, 9, 18, 12, 9, 12, 14], [2.419, 1.3580000000000001, 1.462, 2.1890000000000001, 2.5819999999999999, 2.823, 1.3660000000000001, 1.5519999999999998, 1.6120000000000001, 1.589, 1.885, 2.8849999999999998, 1.2629999999999999, 2.7010000000000001, 1.327, 1.448000

In [89]:
base_dir
datafile = c[1]
datafile
model_dict[datafile]

{u'Columns': {u'motion-param': {u'FileSelector': {u'EntitiesKeys': {u'run': u'run',
     u'subject': u'sub'},
    u'pattern': u'derivatives/mcflirt/par/_runcode_{0[run]:1d}/_subject_id_sub-{0[sub]}/sub-{0[sub]}_task-mixedgamblestask_run-{0[run]}_bold_mcf.nii.gz.par'},
   u'Regressors': [u'all', u'deriv1']},
  u'param-gain': {u'Demean': True,
   u'HRFModelling': u'Gamma+derivs',
   u'Level': u'n/a',
   u'ModulationOrder': 1,
   u'ModulationVar': u'gain',
   u'Variable': u'trial_type',
   u'duration': 1.3932149481304148},
  u'param-loss': {u'Demean': True,
   u'HRFModelling': u'Gamma+derivs',
   u'Level': u'n/a',
   u'ModulationVar': u'loss',
   u'Variable': u'trial_type',
   u'duration': 1.3932149481304148},
  u'param-rt': {u'Demean': True,
   u'HRFModelling': u'Gamma+derivs',
   u'Level': u'n/a',
   u'ModulationVar': u'RT',
   u'Variable': u'trial_type',
   u'duration': 1.3932149481304148},
  u'task': {u'HRFModelling': u'Gamma',
   u'Level': u'n/a',
   u'Variable': u'trial_type',
   u'

In [90]:
#dict_regress = utils.get_run_conditions(base_dir, c[0], model_data[c[0]], verbose=utils.VERB['info'])

In [93]:
utils._get_nipype_contrasts(model_dict[datafile])

[(u'param-loss-neg', 'T', [u'param-loss'], [-1]),
 (u'task>Baseline', 'T', [u'task'], [1]),
 (u'param-gain', 'T', [u'param-gain'], [1])]

In [94]:
data = utils.get_json_model_Ydata(all_jsons[0], level='Run', verbose=utils.VERB['info'])

base dir /home/jb/data/bids/ds005
json_model /home/jb/data/bids/ds005/models/ds-005_type-russ_sub-all_model.json


In [95]:
pp.pprint(data[:min(10, len(data))])

[ '/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-02_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-03_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-01_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-03_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-02_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-02_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-01_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-03_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-12/func/sub-12_task-mixedgamblestask_run-02_bold.nii.gz']


In [101]:
model_pattern = "*_model.json"
model_data = utils.associate_model_data(base_dir, model_pattern, verbose=utils.VERB['none'])
pp.pprint(model_data.keys())

['data_dict', 'base_dir']


In [103]:
datafile = data[0]
model_dict = model_data['data_dict'][datafile]
pp.pprint(model_dict)

{ u'Columns': { u'dummy': { u'HRFModelling': u'Gamma',
                            u'Level': -1,
                            u'Variable': u'respcat'},
                u'gain': { u'HRFModelling': u'Gamma',
                           u'Level': 0,
                           u'ModulationOrder': 1,
                           u'ModulationVar': u'RT',
                           u'Variable': u'respcat'},
                u'loss': { u'HRFModelling': u'Gamma',
                           u'Level': 1,
                           u'ModulationVar': u'RT',
                           u'Variable': u'respcat'}},
  u'Contrasts': { u'pos_vs_neg_contrast': { u'Columns': [u'gain', u'loss'],
                                            u'Weights': [1, -1]}},
  u'DependentVariable': u'task-mixedgamblestask',
  u'Level': u'Run'}


In [72]:
import pandas as pd
tsv_file = datafile.split('_bold.nii')[0] + '_events.tsv'
assert osp.isfile(tsv_file), "{} is not a file ".format(tsv_file)
print(tsv_file)

/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_events.tsv


In [73]:
dic = utils._get_json_dict_from_tsv_file(tsv_file)
#utils._get_tsv_lines(dic, 'trial_type', 'chair')

In [74]:
model_dict = model_data[datafile]
print(model_dict.keys())
regressors = model_dict['Columns']
pp.pprint(regressors)

[u'Contrasts', u'DependentVariable', u'Columns', u'Level']
{ u'dummy': { u'HRFModelling': u'Gamma', u'Level': -1, u'Variable': u'respcat'},
  u'gain': { u'HRFModelling': u'Gamma',
             u'Level': 0,
             u'ModulationOrder': 1,
             u'ModulationVar': u'RT',
             u'Variable': u'respcat'},
  u'loss': { u'HRFModelling': u'Gamma',
             u'Level': 1,
             u'ModulationVar': u'RT',
             u'Variable': u'respcat'}}


In [107]:
list_dicts, list_reg, log = utils.get_run_conditions(base_dir, datafile, model_dict)
condition_names = list_dicts.keys()

In [77]:
#pp.pprint(list_dicts)
for datafile in data[:min(10, len(data))]:
    #print(datafile)
    try: 
        nipype_info = utils.get_nipype_run_info(datafile, model_dict, verbose=utils.VERB['none'])
    except:
        print(datafile, ' : has a pb ... a condition has no event ')

/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_bold.nii.gz  : has a pb ... a condition has no event 
/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-02_bold.nii.gz  : has a pb ... a condition has no event 
/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-03_bold.nii.gz  : has a pb ... a condition has no event 
/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-01_bold.nii.gz  : has a pb ... a condition has no event 
/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-03_bold.nii.gz  : has a pb ... a condition has no event 
/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-02_bold.nii.gz  : has a pb ... a condition has no event 
/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-02_bold.nii.gz  : has a pb ... a condition has no event 
/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-01_bold.nii.gz  : has a pb ... a conditio

In [78]:
#print(datafile)
#pp.pprint(nipype_info)
#print(nipype_info)

In [110]:
dict_regressors, dict_other,  _log = \
                utils.get_run_conditions(base_dir, datafile, model_dict)#, verbose=verbose)
condition_names = dict_regressors.keys() 
#print(dict_regressors)
bunch = utils.make_nipype_bunch(dict_regressors, dict_other, bunch_type='spm')

In [111]:
bunch

Bunch(conditions=[u'dummy', u'gain', u'loss'], durations=[[3], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]], onsets=[[0.0], [8.0, 74.0, 82.0, 110.0, 120.0, 134.0, 148.0, 156.0, 178.0, 342.0, 346.0, 382.0, 386.0, 402.0, 424.0, 440.0, 458.0], [4.0, 18.0, 24.0, 28.0, 38.0, 42.0, 46.0, 52.0, 56.0, 62.0, 78.0, 86.0, 90.0, 94.0, 98.0, 102.0, 106.0, 116.0, 126.0, 130.0, 140.0, 160.0, 164.0, 170.0, 174.0, 182.0, 192.0, 202.0, 208.0, 212.0, 220.0, 224.0, 234.0, 238.0, 246.0, 250.0, 256.0, 260.0, 266.0, 272.0, 280.0, 284.0, 296.0, 310.0, 316.0, 320.0, 324.0, 330.0, 336.0, 350.0, 360.0, 364.0, 368.0, 374.0, 378.0, 398.0, 408.0, 412.0, 416.0, 420.0, 428.0, 436.0, 444.0, 448.0, 462.0, 466.0, 470.0, 474.0]], pmod=[[], [Bunch(name=u'RT', param=[1.6369999999999998, 1.8220000000000001, 1.206,

### copy a ds tree with only the json and tsv files

In [83]:
source_dir = '/home/jb/data/bids/ds005'
dest_dir = '/home/jb/code/bids2pype/bids2pype/tests/data/ds005'
# utils.create_empty_bids(source_dir, dest_dir, ['*.json','*.tsv'])

In [ ]:
utils._sorted_by_depth('/home/jb/data/bids/ds105', "*sub*_model.json")

In [ ]:
res = utils._get_substr_between(datafile, 'task-', '_')
print(res)